# Propensity Modeling

In [3]:
%%bash
pip3 install user_agents
pip3 install tqdm

  Running setup.py bdist_wheel for user-agents: started
  Running setup.py bdist_wheel for user-agents: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/4c/36/d8/6b28dfb933d4397e28dcf7b6d426b85808a22eed8c4107e6c2
Successfully built user-agents


In [4]:
import numpy as np
import pandas as pd
import pickle
import urllib
from tqdm import tqdm, trange, tqdm_pandas
import os
from user_agents import parse
import datetime
import time
from sklearn import linear_model
import tensorflow as tf
# pd.set_option('display.max_columns', 1000)

## Load Data:

In [5]:
%%time
with open( "../Data/Feature_Engineering/df_features_dummies.p", "rb" ) as f:
    df_features_dummies = pickle.load(f)

with open( "../Data/Feature_Engineering/df_label.p", "rb" ) as f:
    df_label = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Feature_Engineering/df_features_dummies.p'

In [6]:
%%time
X = df_features_dummies
Y = df_label

NameError: name 'df_features_dummies' is not defined

In [7]:
print(X.shape)
bar_80 = np.floor(X.shape[0]*0.8).astype(int)
bar_80=bar_80+1
bar_80

NameError: name 'X' is not defined

In [ ]:
X_train=X[:bar_80]
Y_train=Y[:bar_80]
X_test=X[bar_80:]
Y_test=Y[bar_80:]

In [ ]:
%%time
X_train=tf.cast(X_train, tf.float32) 
Y_train=tf.cast(Y_train, tf.float32) 

X_test=tf.cast(X_test, tf.float32) 
Y_test=tf.cast(Y_test, tf.float32) 

In [ ]:
sess = tf.InteractiveSession()

In [1]:
X_train.eval()
tf.cast(X_train, tf.float32)

NameError: name 'X_train' is not defined

## Save 


In [10]:
%%time
with open( "../Data/Propensity_Modeling/X_train.p", "wb" ) as f:
    pickle.dump(X_train, f)
with open( "../Data/Propensity_Modeling/Y_train.p", "wb" ) as f:
    pickle.dump(Y_train, f)
with open( "../Data/Propensity_Modeling/X_test.p", "wb" ) as f:
    pickle.dump(X_test, f)
with open( "../Data/Propensity_Modeling/Y_test.p", "wb" ) as f:
    pickle.dump(Y_test, f)

TypeError: can't pickle _thread.lock objects

## 1. Logistic Regression Model:

In [49]:
logreg=linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, 
                                        intercept_scaling=1, class_weight=None, random_state=1, solver='liblinear', max_iter=100, 
                                        multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)

In [50]:
# %%time
# logreg.fit(X_train, Y_train)

### Predict confidence scores for samples:

In [51]:
# Y_prob=logreg.decision_function(X)

In [52]:
Y_prob

NameError: name 'Y_prob' is not defined

### Log of probability estimates:

In [53]:
Y_logProb=logreg.predict_log_proba(X_train)

NotFittedError: Call fit before prediction

In [54]:
print(len(np.exp(Y_logProb)))
print(np.exp(Y_logProb))
print('first column is prob of choosing label 0, second column is the prob of choosing label 1')

NameError: name 'Y_logProb' is not defined

### Predicted labels:

In [55]:
Y_pred = logreg.predict(X_test)

NotFittedError: This LogisticRegression instance is not fitted yet

In [56]:
Y_pred

NameError: name 'Y_pred' is not defined

In [57]:
sum(Y_pred==Y_test)/Y_test.shape[0]

NameError: name 'Y_pred' is not defined

## 2. Neuron Network:

### 0. Data assignment

In [32]:
%%time
print('shape of X_train',X_train.shape)
print('shape of Y_test',Y_test.shape)

shape of X_train (176038, 2362)
shape of Y_test (44009,)
CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 1.25 ms


In [33]:
%%time
X_train=tf.reshape(X_train,[X_train.shape[0],X_train.shape[1]])
X_test=tf.reshape(X_test,[X_test.shape[0],X_test.shape[1]])

Y_train=tf.reshape(Y_train,[Y_train.shape[0],X_train.shape[1]])
Y_test=tf.reshape(Y_test,[Y_test.shape[0],1])

TypeError: Failed to convert object of type <class 'list'> to Tensor. Contents: [Dimension(176038), Dimension(2362)]. Consider casting elements to a supported type.

In [21]:
print('shape of Y_test',Y_test.shape)

shape of Y_test (44010,)


In [22]:
(nrTrainSamples, dimX) = X_train.shape
(nrTestSamples,dimY)  = Y_test.shape

print ("Finished: data loaded. Stats below: ")
print ("Number of training samples: %d" %nrTrainSamples)
print ("Number of testing  samples: %d" %nrTestSamples)
print ("Dimension of X: %d" %dimX)
print ("Dimension of Y: %d" %dimY)
IMAGE_DIM = int(dimX**0.5) 

ValueError: not enough values to unpack (expected 2, got 1)

### 1. Basic Neuron Setting

In [80]:
sess = tf.InteractiveSession()
BATCH_SIZE    = 100
MAX_ITERS     = 200
LEARNING_RATE = 0.1
n_hidden=256  # 256 units in this hidden lapyer

### 2. Model on data

In [81]:
# ---------------------------------------------------------------
# Linear layer:
def LinearLayer(x,dim1,dim2):
    W=tf.Variable(tf.random_normal([dim1,dim2],stddev=0.01))
    b=tf.Variable(tf.random_normal([1, dim2],stddev=0.01))
    return tf.matmul(x,W)+b

# ---------------------------------------------------------------    
# ReLU hidden layer:
def ReLULayer(x,dim1,dim2,dropout_prob=0.5):
    W_h = tf.Variable(tf.truncated_normal([dim1,dim2], stddev=0.1))
    b_h = tf.constant(0.1, shape=[dim2])
    z_h = tf.matmul(x,W_h) + b_h
    y_h = tf.nn.relu(z_h)
    
    # Dropout layer
    output = tf.nn.dropout(y_h, dropout_prob)
    return output

# ---------------------------------------------------------------    

# Model has 2 hidden ReLU layers.
def Model(x,dim1,dim2):
    # apply dropout to input layer
    prob_keep_input=0.8
    x=tf.nn.dropout(x,prob_keep_input)
    
    dimH1, dimH2=n_hidden, n_hidden
    
    # hidden layer 1:
    h1=ReLULayer(x,dim1,dimH1,dropout_prob=0.5)
    # hidden layer 2:
    h2=ReLULayer(h1,dimH1,dimH2,dropout_prob=0.5)
    
    
    # output layer (Softmax layer):
    z=LinearLayer(h2,dimH2,dim2)
    y_=tf.nn.softmax(z)
    # y_ is the predicted label
    
    return [y_, z]

# ---------------------------------------------------------------
# Generic training function:
# ---------------------------------------------------------------
# x                   tf.palceholder or INPUT
# y                   tf.palceholder or OUTPUT
# train_op            training operation (tf.Optimizer)
# accuracy     
# trainX, trainY      training data and labels
# testX, testY        testing data and labels

def trainModel(accuracy, train_op, x, y, trainX, trainY, testX, testY):
    # initialize variables
    tf.global_variables_initializer().run()

    print ("===============================")
    print ("Training started........")

    for indexIter in range(MAX_ITERS):
        for startIndex, endIndex in zip( range(0,len(trainX),BATCH_SIZE), range(BATCH_SIZE,len(trainX),BATCH_SIZE)):
            sess.run(train_op, feed_dict={x: trainX[startIndex:endIndex], y: trainY[startIndex:endIndex]})

        # Visualize accuracy only every 10 iterations
        if indexIter%10==0 :

            acc_train = sess.run(accuracy, feed_dict={x:trainX, y:trainY})
            acc_test  = sess.run(accuracy, feed_dict={x:testX,  y:testY})

            print('Iteration %d: Accuracy %.5f(training) %.5f(testing)' %(indexIter, acc_train, acc_test))
    
    print('Iteration %d: Accuracy %.5f(training) %.5f(testing)' %(indexIter, acc_train, acc_test))
    print ("Training finished.")
    print ("===============================")




In [82]:
# ---------------------------------------------------------------  
def main():
    
    #============================
    # 1. Build model
    #============================
    
    x = tf.placeholder(tf.float32, shape=[None, dimX]) 
    # None is the number of data set, 784 is the dimensions of features
    y = tf.placeholder(tf.float32, shape=[None, dimY])
    
    # model returns a score vector over all classes
    model = Model(x, dimX, dimY)
    
    # Get predicted classes
    predict_op = tf.argmax(model[0], 1)
    
    # Loss function (Cross-entropy for classification)
    cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits = model[1]))
    # Error: Only call `softmax_cross_entropy_with_logits` with named arguments (labels=..., logits=..., ...)
    
    # Accuracy(if different measure from loss)
    correct_prediction = tf.equal(predict_op, tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    #============================
    # 2. Train model
    #============================
    train_op = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy)
    trainModel(accuracy, train_op, x, y, X_train, Y_train, X_test, Y_test)

In [84]:
#(REPRODUCIBILITY) set random seeds
tf.set_random_seed(722)

main()
  
# y_true = np.argmax(mnist.test.labels,1)
    
# Save the the whole session
# saver = tf.train.Saver()
# name_of_checkpoint = "model_C"
# print("\n Saving model to: " + name_of_checkpoint)
# saver.save(sess,name_of_checkpoint)

TypeError: Failed to convert object of type <class 'list'> to Tensor. Contents: [256, Dimension(1)]. Consider casting elements to a supported type.